# 1. Installing PySpark in Google Colab


In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [872 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,231 kB in 2s (2,150 kB/s)
Readi

# 2. Reading Data


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("our first session") \
    .config("spark.sql.sessionEncoding", "UTF-8") \
    .getOrCreate()

df = spark.read.csv("googlePatents.csv", header=True, encoding="UTF-8")

df.show()


+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|              id|               title|                           assignee|                inventor/author|priority date|filing/creation date|publication date|grant date|
+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|  CN-114151200-A|            Igniter |         中国航发四川燃气涡轮研究院|  魏豪, 时远, 何鹏, 赵弦, 袁...|   2021-10-20|          2021-10-20|      2022-03-08|      NULL|
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|                     Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce North...|           Michael John Head...|   2020-01-14|          2020-01-14|      2021-07-15|      NU

In [ ]:
# prompt: je veux savoir le nombre de ligne de fichier csv google patent

count = df.count()
print(f"Number of rows in the google patent csv file: {count}")


Number of rows in the google patent csv file: 155555


In [ ]:
#Viewing the dataframe schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- inventor/author: string (nullable = true)
 |-- priority date: string (nullable = true)
 |-- filing/creation date: string (nullable = true)
 |-- publication date: string (nullable = true)
 |-- grant date: string (nullable = true)



In [ ]:

f_df = df.filter(df['assignee'] == '中国航发四川燃气涡轮研究院')
f_df.show()

+--------------+--------------------+--------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|            id|               title|                  assignee|                inventor/author|priority date|filing/creation date|publication date|grant date|
+--------------+--------------------+--------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|CN-114151200-A|            Igniter |中国航发四川燃气涡轮研究院|  魏豪, 时远, 何鹏, 赵弦, 袁...|   2021-10-20|          2021-10-20|      2022-03-08|      NULL|
|CN-116677498-B|Novel hypersonic ...|中国航发四川燃气涡轮研究院|桂丰, 马存祥, 钟世林, 康玉东...|   2023-08-03|          2023-08-03|      2023-10-17|2023-10-17|
|CN-116374179-A|Series hybrid ele...|中国航发四川燃气涡轮研究院|   伏宇, 王孝军, 刘昭威, 郑天慧|   2023-06-05|          2023-06-05|      2023-07-04|      NULL|
|CN-108561898-B|Coaxial partition...|中国航发四川燃气涡轮研究院|  桂韬, 房人麟, 邱伟, 黄兵, ...|   2017-12-27|          2017-12-27| 

In [ ]:
f_df = df.filter(df['title'] == 'Igniter')
f_df.show()
#we have problem with filtering title sence we have chines language ..

+---+-----+--------+---------------+-------------+--------------------+----------------+----------+
| id|title|assignee|inventor/author|priority date|filing/creation date|publication date|grant date|
+---+-----+--------+---------------+-------------+--------------------+----------------+----------+
+---+-----+--------+---------------+-------------+--------------------+----------------+----------+



In [ ]:
from pyspark.sql.functions import col
filtered_df = df.filter(~(col('id').startswith('JP') | col('id').startswith('CN')))
filtered_df.show(truncate=False)
total_rows = filtered_df.count()
print("Nombre total de lignes dans filtered_df :", total_rows)

+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------+-----------------------------------------------------------------------------------------------------------+-------------+--------------------+----------------+----------+
|id              |title                                                                                                                  |assignee                                        |inventor/author                                                                                            |priority date|filing/creation date|publication date|grant date|
+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------+-----------------------------------------------------------------------------------------------

In [ ]:
import random
from pyspark.sql.functions import lit
from pyspark.sql import functions as F

# Define the sources and their respective weights
sources = ["uspto", "google patent", "wipo", "Espace net"]
weights = [0.2, 0.5, 0.15, 0.15]  # Corresponding weights for each source
random.seed(123)
# Create a function to randomly select a source based on weights
def random_source():
    return random.choices(sources, weights)[0]

# Define a Spark UDF (User Defined Function) to apply the random source selection
random_source_udf = F.udf(random_source)

# Add a new column with the randomly chosen source to the DataFrame
filtered_df = filtered_df.withColumn("source_scrape", random_source_udf())

# Show the DataFrame with the new column
filtered_df.show(truncate=True)


+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+
|              id|               title|                           assignee|     inventor/author|priority date|filing/creation date|publication date|grant date|source_scrape|
+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|google patent|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce North...|Michael John Head...|   2020-01-14|          2020-01-14|      2021-07-15|      NULL|   Espace net|
|   US-7966830-B2|Fuel cell/combust...|                 The Boeing Company|    David L. Daggett|   2006-06-29|          2006-06-29

In [ ]:
from pyspark.sql.functions import col, substring
filtered_df = filtered_df.withColumn("country", substring(col("id"), 1, 2))
filtered_df.show(truncate=False)

+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------+-----------------------------------------------------------------------------------------------------------+-------------+--------------------+----------------+----------+-------------+-------+
|id              |title                                                                                                                  |assignee                                        |inventor/author                                                                                            |priority date|filing/creation date|publication date|grant date|source_scrape|country|
+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------+---------------------------------------------------

In [ ]:
from pyspark.sql.functions import col, substring
from pyspark.sql.functions import col, substring, udf
from pyspark.sql.types import StringType

# Dictionnaire de correspondance des abréviations aux noms complets des pays
pays_complets = {
    'LT': 'Lithuania',
    'FI': 'Finland',
    'UA': 'Ukraine',
    'EA': 'European Economic Area (EEA)',
    'RO': 'Romania',
    'NL': 'Netherlands',
    'EP': 'Europe (European patents)',
    'PL': 'Poland',
    'MX': 'Mexico',
    'AT': 'Austria',
    'RU': 'Russia',
    'HR': 'Croatia',
    'CZ': 'Czech Republic',
    'PT': 'Portugal',
    'WO': 'World Intellectual Property Organization (WIPO)',
    'TW': 'Taiwan',
    'OA': 'Africa (African patents)',
    'SU': 'Soviet Union',
    'DD': 'East Germany',
    'AU': 'Australia',
    'SA': 'Saudi Arabia',
    'CA': 'Canada',
    'GB': 'United Kingdom',
    'BR': 'Brazil',
    'DE': 'Germany',
    'ES': 'Spain',
    'IL': 'Israel',
    'TR': 'Turkey',
    'ZA': 'South Africa',
    'KR': 'South Korea',
    'US': 'United States',
    'FR': 'France',
    'CH': 'Switzerland',
    'SG': 'Singapore',
    'IT': 'Italy',
    'SE': 'Sweden',
    'BG': 'Bulgaria',
    'PH': 'Philippines',
    'SK': 'Slovakia',
    'LV': 'Latvia',
    'TH': 'Thailand',
    'CS': 'Czechoslovakia',
    'NZ': 'New Zealand',
    'HU': 'Hungary',
    'SI': 'Slovenia',
    'IE': 'Ireland',
    'BE': 'Belgium',
    'MA': 'Morocco',
    'NO': 'Norway',
    'AR': 'Argentina',
    'LU': 'Luxembourg',
    'DK': 'Denmark',
    'GR': 'Greece'
}


# Fonction pour remplacer les abréviations par les noms complets des pays
def replace_country_abbreviations(country_abbrev):
    return pays_complets.get(country_abbrev, country_abbrev)

# Créer une fonction UDF (User Defined Function)
replace_country_abbreviations_udf = udf(replace_country_abbreviations)

# Remplacer les abréviations dans la colonne "country" par les noms complets des pays
filtered_df = filtered_df.withColumn("country_full", replace_country_abbreviations_udf(col("country")))

# Afficher le DataFrame avec les noms complets des pays
filtered_df.show(truncate=False)

+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------+-----------------------------------------------------------------------------------------------------------+-------------+--------------------+----------------+----------+-------------+-------+-------------------------+
|id              |title                                                                                                                  |assignee                                        |inventor/author                                                                                            |priority date|filing/creation date|publication date|grant date|source_scrape|country|country_full             |
+----------------+-----------------------------------------------------------------------------------------------------------------------+------------------------------------------------

In [ ]:
from pyspark.sql.functions import datediff

filtered_df = filtered_df.withColumn("grant_to_filing_duration_days",
                                     datediff("grant date", "filing/creation date"))

filtered_df.show()


+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+-------+--------------------+-----------------------------+
|              id|               title|                           assignee|     inventor/author|priority date|filing/creation date|publication date|grant date|source_scrape|country|        country_full|grant_to_filing_duration_days|
+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+-------+--------------------+-----------------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|google patent|     US|       United States|                         1329|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce No

In [ ]:
filtered_df = filtered_df.withColumn("filing_to_priority_duration_days",
                                     datediff(col("`filing/creation date`"), col("`priority date`")))

filtered_df.show()


+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+
|              id|               title|                           assignee|     inventor/author|priority date|filing/creation date|publication date|grant date|source_scrape|country|        country_full|grant_to_filing_duration_days|filing_to_priority_duration_days|
+----------------+--------------------+-----------------------------------+--------------------+-------------+--------------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|        uspto|     US|       United States|

In [ ]:
# Rename the column "filing/creation date" to "creation date"
filtered_df = filtered_df.withColumnRenamed("filing/creation date", "creation_date")

# Show the DataFrame with the renamed column
filtered_df.show()


+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+
|              id|               title|                           assignee|     inventor/author|priority date|creation_date|publication date|grant date|source_scrape|country|        country_full|grant_to_filing_duration_days|filing_to_priority_duration_days|
+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12|   2014-05-28|      2018-01-16|2018-01-16|google patent|     US|       United States|                         132

In [ ]:
filtered_df = filtered_df.withColumnRenamed("priority date", "priority_date")
filtered_df = filtered_df.withColumnRenamed("inventor/author", "inventor")
filtered_df = filtered_df.withColumnRenamed("publication date", "publication_date")
filtered_df = filtered_df.withColumnRenamed("grant date", "grant_date")




In [ ]:
filtered_df = filtered_df.withColumn("creation_to_publication_duration_days",
                                     datediff(col("publication_date"), col("creation_date")))

filtered_df.show()


+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+-------------------------------------+
|              id|               title|                           assignee|            inventor|priority_date|creation_date|publication_date|grant_date|source_scrape|country|        country_full|grant_to_filing_duration_days|filing_to_priority_duration_days|creation_to_publication_duration_days|
+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+-------------------------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12

In [ ]:

total_rows = filtered_df.count()
print("Nombre total de lignes dans filtered_df_with_names :", total_rows)


Nombre total de lignes dans filtered_df_with_names : 102606


In [ ]:
filtered_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- inventor: string (nullable = true)
 |-- priority_date: string (nullable = true)
 |-- creation_date: string (nullable = true)
 |-- publication_date: string (nullable = true)
 |-- grant_date: string (nullable = true)
 |-- source_scrape: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_full: string (nullable = true)
 |-- grant_to_filing_duration_days: integer (nullable = true)
 |-- filing_to_priority_duration_days: integer (nullable = true)
 |-- creation_to_publication_duration_days: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import col, isnan, when, count

# Iterate over each column in the DataFrame
for column in filtered_df.columns:
    # Count the number of null values in the column
    null_count = filtered_df.where(col(column).isNull()).count()
    # Print the column name and the number of null values
    print(f"Column '{column}' has {null_count} null values")

# Alternatively, you can use the 'isNull' function with 'select' and 'alias' to create a DataFrame with null counts
null_counts_df = filtered_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in filtered_df.columns])
# Show the DataFrame with null counts
null_counts_df.show()


Column 'id' has 0 null values
Column 'title' has 0 null values
Column 'assignee' has 75 null values
Column 'inventor' has 346 null values
Column 'priority_date' has 129 null values
Column 'creation_date' has 33 null values
Column 'publication_date' has 9 null values
Column 'grant_date' has 26224 null values
Column 'source_scrape' has 0 null values
Column 'country' has 0 null values
Column 'country_full' has 0 null values
Column 'grant_to_filing_duration_days' has 26255 null values
Column 'filing_to_priority_duration_days' has 169 null values
Column 'creation_to_publication_duration_days' has 48 null values
+---+-----+--------+--------+-------------+-------------+----------------+----------+-------------+-------+------------+-----------------------------+--------------------------------+-------------------------------------+
| id|title|assignee|inventor|priority_date|creation_date|publication_date|grant_date|source_scrape|country|country_full|grant_to_filing_duration_days|filing_to_prio

In [ ]:
from pyspark.sql.functions import when

# Handle missing values for 'assignee' and 'inventor' columns
filled_df = filtered_df.fillna({'assignee': 'unknown', 'inventor': 'unknown'})

# Handle missing values for date columns ('priority_date', 'creation_date', 'publication_date', 'grant_date')
# Assuming 'unknown' isn't suitable for date columns, we can replace null values with a placeholder date or leave them as null
# Let's replace null values with a placeholder date '1900-01-01' for demonstration purposes
date_columns = ['priority_date', 'creation_date', 'publication_date', 'grant_date']
for column in date_columns:
    filled_df = filled_df.withColumn(column, when(col(column).isNull(), '1900-01-01').otherwise(col(column)))

# Handle missing values for other columns (numeric columns)
# We can replace null values with 0 for numeric columns ('grant_to_filing_duration_days', 'filing_to_priority_duration_days', 'creation_to_publication_duration_days')
numeric_columns = ['grant_to_filing_duration_days', 'filing_to_priority_duration_days', 'creation_to_publication_duration_days']
for column in numeric_columns:
    filled_df = filled_df.fillna({column: 111111})

# Verify the changes
filled_df.show()


+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+-------------------------------------+
|              id|               title|                           assignee|            inventor|priority_date|creation_date|publication_date|grant_date|source_scrape|country|        country_full|grant_to_filing_duration_days|filing_to_priority_duration_days|creation_to_publication_duration_days|
+----------------+--------------------+-----------------------------------+--------------------+-------------+-------------+----------------+----------+-------------+-------+--------------------+-----------------------------+--------------------------------+-------------------------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|   2008-02-12

In [ ]:
# Define the file path for saving the CSV file
output_path = "goldS2.csv"

# Save the DataFrame to a CSV file
filled_df.write.csv(output_path, header=True, mode="overwrite")


In [ ]:
from google.colab import drive
import shutil
import pandas as pd
import pyspark.sql.functions as F

# Montez votre Google Drive
drive.mount('/content/drive')

# Convertir le DataFrame Spark en DataFrame Pandas

pandas_df = filtered_df.toPandas()

# Enregistrer le DataFrame Pandas au format Excel
pandas_df.to_csv("/content/drive/MyDrive/goldS2.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 5: Stop Spark session (optional, typically done at the end of the program)
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Snowflake Schema") \
    .getOrCreate()

# Load your data into a DataFrame, specifying the UTF-8 encoding
df = spark.read.csv("goldS2.csv", header=True, inferSchema=True, encoding="utf-8")

# Define columns for fact table (numeric and foreign key columns)
fact_columns = ["id", "grant_to_filing_duration_days", "filing_to_priority_duration_days", "creation_to_publication_duration_days"]

# Extract fact table data
fact_df = df.select(*[col(column) for column in fact_columns])

# Define columns for dimension tables (qualitative values)
dimension_columns = ["title", "assignee", "inventor", "priority_date", "creation_date", "publication_date", "grant_date", "source_scrape", "country", "country_full"]

# Extract dimension table data
dimension_df1 = df.select(*[col(column) for column in ["id", "title", "assignee", "inventor"]])
dimension_df2 = df.select(*[col(column) for column in ["id", "priority_date", "creation_date", "publication_date", "grant_date"]])
dimension_df3 = df.select(*[col(column) for column in ["id", "source_scrape", "country", "country_full"]])

# Save fact table to CSV file, specifying the UTF-8 encoding
fact_df.write.csv("/content/fact_table.csv", header=True, mode="overwrite", encoding="utf-8")

# Save dimension tables to CSV files, specifying the UTF-8 encoding
dimension_df1.write.csv("/content/dimension_table1.csv", header=True, mode="overwrite", encoding="utf-8")
dimension_df2.write.csv("/content/dimension_table2.csv", header=True, mode="overwrite", encoding="utf-8")
dimension_df3.write.csv("/content/dimension_table3.csv", header=True, mode="overwrite", encoding="utf-8")



In [ ]:
fact_df.show()

+----------------+-----------------------------+--------------------------------+-------------------------------------+
|              id|grant_to_filing_duration_days|filing_to_priority_duration_days|creation_to_publication_duration_days|
+----------------+-----------------------------+--------------------------------+-------------------------------------+
|   US-9869277-B2|                       1329.0|                          2297.0|                               1329.0|
|US-2021215100-A1|                         NULL|                             0.0|                                548.0|
|   US-7966830-B2|                       1825.0|                             0.0|                               1825.0|
|    GB-2602936-A|                         NULL|                             0.0|                                 91.0|
|   EP-2086834-B1|                       1339.0|                           362.0|                               1339.0|
|  US-11572840-B2|                      

In [ ]:
dimension_df1.show()

+----------------+--------------------+-----------------------------------+--------------------+
|              id|               title|                           assignee|            inventor|
+----------------+--------------------+-----------------------------------+--------------------+
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|          Todd Foret|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce North...|Michael John Head...|
|   US-7966830-B2|Fuel cell/combust...|                 The Boeing Company|    David L. Daggett|
|    GB-2602936-A|  Propulsion system |                   Derwent Tech Ltd|Harold Speak Trev...|
|   EP-2086834-B1|Propulsion device...|               Airbus Operations...|Andreas Westenberger|
|  US-11572840-B2|Multi-mode combus...|               General Electric ...|Joel Meier Haynes...|
|KR-20220002857-A|Aircraft prime mo...|쥐케이엔 에어로스페이스 서비시즈...|       사이먼 테일러|
|US-2019293024-A1|Hydrogen-fueled s...|            

In [ ]:
dimension_df2.show()

+----------------+-------------+-------------+----------------+----------+
|              id|priority_date|creation_date|publication_date|grant_date|
+----------------+-------------+-------------+----------------+----------+
|   US-9869277-B2|   2008-02-12|   2014-05-28|      2018-01-16|2018-01-16|
|US-2021215100-A1|   2020-01-14|   2020-01-14|      2021-07-15|      NULL|
|   US-7966830-B2|   2006-06-29|   2006-06-29|      2011-06-28|2011-06-28|
|    GB-2602936-A|   2022-04-20|   2022-04-20|      2022-07-20|      NULL|
|   EP-2086834-B1|   2006-11-29|   2007-11-26|      2011-07-27|2011-07-27|
|  US-11572840-B2|   2019-12-03|   2019-12-03|      2023-02-07|2023-02-07|
|KR-20220002857-A|   2018-10-15|   2019-10-15|      2022-01-07|      NULL|
|US-2019293024-A1|   2018-02-23|   2019-02-25|      2019-09-26|      NULL|
|   US-9291124-B2|   2006-12-18|   2014-03-06|      2016-03-22|2016-03-22|
|   RU-2462397-C2|   2006-11-29|   2007-11-28|      2012-09-27|2012-09-27|
|  US-11885260-B2|   2020

In [ ]:
dimension_df3.show()

+----------------+-------------+-------+--------------------+
|              id|source_scrape|country|        country_full|
+----------------+-------------+-------+--------------------+
|   US-9869277-B2|google patent|     US|       United States|
|US-2021215100-A1|   Espace net|     US|       United States|
|   US-7966830-B2|        uspto|     US|       United States|
|    GB-2602936-A|google patent|     GB|      United Kingdom|
|   EP-2086834-B1|   Espace net|     EP|Europe (European ...|
|  US-11572840-B2|         wipo|     US|       United States|
|KR-20220002857-A|google patent|     KR|         South Korea|
|US-2019293024-A1|   Espace net|     US|       United States|
|   US-9291124-B2|         wipo|     US|       United States|
|   RU-2462397-C2|google patent|     RU|              Russia|
|  US-11885260-B2|google patent|     US|       United States|
|   US-8434292-B2|        uspto|     US|       United States|
|   US-7690192-B2|         wipo|     US|       United States|
|   EP-2

In [ ]:
from google.colab import drive
import shutil

# Montez votre Google Drive
drive.mount('/content/drive')

# Spécifiez le dossier de destination dans votre Google Drive
destination_folder = "/content/drive/My Drive/analysis"

# Déplacez les fichiers CSV vers le dossier de destination dans votre Google Drive
shutil.move("/content/fact_table.csv", destination_folder)
shutil.move("/content/dimension_table1.csv", destination_folder)
shutil.move("/content/dimension_table2.csv", destination_folder)
shutil.move("/content/dimension_table3.csv", destination_folder)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/analysis/dimension_table3.csv'

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths to your CSV files
csv_files = ["/content/fact_table.csv", "/content/dimension_table1.csv", "/content/dimension_table2.csv", "/content/dimension_table3.csv"]

# Specify the destination folder in your Google Drive
destination_folder = "/content/drive/My Drive/patents/"

# Create the destination folder if it doesn't exist
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Move CSV files to the destination folder
for file_path in csv_files:
    shutil.move(file_path, destination_folder)

print("CSV files moved to Google Drive successfully.")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV files moved to Google Drive successfully.


In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Patent NLP") \
    .getOrCreate()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f854173a521e284a11ae95f3380b93b59f0d40e9bae4f4bcf57ba049d4e03945
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
spark = SparkSession.builder \
    .appName("Patent NLP") \
    .getOrCreate()

# Step 2: Load Data into Spark
file_path = "updated_data(1).xlsx"

# Ensure you have the necessary dependencies for reading Excel files
# If needed, you can install the package using:
# !pip install spark-excel

df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)


Py4JJavaError: An error occurred while calling o36.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.crealytics.spark.excel. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessor

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import random

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Patent Technology Sector Assignment") \
    .getOrCreate()

# Step 1: Load Data into Spark
file_path = "/mnt/data/updated_data(1).xlsx"

df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

# Step 2: Define a list of possible technology sectors
technology_sectors = ["Software", "Biotechnology", "Electronics", "Healthcare", "Automotive", "Telecom"]

# Step 3: Create a UDF to assign random technology sectors
def random_technology_sector():
    return random.choice(technology_sectors)

random_technology_sector_udf = udf(random_technology_sector, StringType())

# Step 4: Add a new column with random technology sectors
df = df.withColumn("technology_sector", random_technology_sector_udf())

# Step 5: Select only the patent ID and the new technology sector columns
df_result = df.select(col("id"), col("technology_sector"))

# Step 6: Save the resulting DataFrame as a new CSV file
output_path = "/mnt/data/patents_with_random_technology_sector.csv"
df_result.write.csv(output_path, header=True)

# Stop the Spark session
spark.stop()


Py4JJavaError: An error occurred while calling o50.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.crealytics.spark.excel. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessor

In [ ]:
%pip install pyspark
%pip install openpyxl
%pip install com.crealytics:spark-excel_2.12:0.13.5


ERROR: Invalid requirement: 'com.crealytics:spark-excel_2.12:0.13.5'


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("our first session") \
    .config("spark.sql.sessionEncoding", "UTF-8") \
    .getOrCreate()

df = spark.read.csv("googlePatents.csv", header=True, encoding="UTF-8")

df.show()


+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|              id|               title|                           assignee|                inventor/author|priority date|filing/creation date|publication date|grant date|
+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+
|  CN-114151200-A|            Igniter |         中国航发四川燃气涡轮研究院|  魏豪, 时远, 何鹏, 赵弦, 袁...|   2021-10-20|          2021-10-20|      2022-03-08|      NULL|
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|                     Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce North...|           Michael John Head...|   2020-01-14|          2020-01-14|      2021-07-15|      NU

In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- inventor/author: string (nullable = true)
 |-- priority date: string (nullable = true)
 |-- filing/creation date: string (nullable = true)
 |-- publication date: string (nullable = true)
 |-- grant date: string (nullable = true)



In [ ]:
# df = spark.read.csv("googlePatents.csv", header=True, encoding="UTF-8")

# # Define a list of possible technology sectors
# technology_sectors = ["Software", "Biotechnology", "Electronics", "Healthcare", "Automotive", "Telecom"]

# # Create a UDF to assign random technology sectors
# def random_technology_sector():
#     return random.choice(technology_sectors)

# random_technology_sector_udf = udf(random_technology_sector, StringType())

# # Add a new column with random technology sectors
# df_with_technology_sector = df.withColumn("technology_sector", random_technology_sector_udf())

# # Display the DataFrame
# df_with_technology_sector.show()
# Step 1: Set Up Spark Environment with Excel Dependency
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Initialize Spark session with the Excel library dependency
spark = SparkSession.builder \
    .appName("Patent NLP") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.5") \
    .getOrCreate()

# Step 2: Load Data into Spark
file_path = "/mnt/data/updated_data(1).xlsx"

df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_path)

# Step 3: Detect and Tokenize Text Data Based on Language
import nltk
from nltk.tokenize import word_tokenize
import jieba
from janome.tokenizer import Tokenizer as JanomeTokenizer
from konlpy.tag import Okt

# Download NLTK data
nltk.download('punkt')

# Define UDF for language detection
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

detect_language_udf = udf(detect_language, StringType())
df = df.withColumn("language", detect_language_udf(df.title))

# Define UDFs for tokenization
def tokenize(text, lang):
    if lang == 'zh-cn':  # Chinese
        return ' '.join(jieba.cut(text))
    elif lang == 'ja':  # Japanese
        janome_tokenizer = JanomeTokenizer()
        return ' '.join([token.surface for token in janome_tokenizer.tokenize(text)])
    elif lang == 'ko':  # Korean
        okt = Okt()
        return ' '.join(okt.morphs(text))
    else:  # Default to English
        return ' '.join(word_tokenize(text.lower()))

def tokenize_combined(text, lang):
    return tokenize(text, lang)

tokenize_udf = udf(tokenize_combined, StringType())
df = df.withColumn("tokenized_title", tokenize_udf(df.title, df.language))

# Step 4: Extract Technology Sector and Industry using NLP
def determine_sector(title):
    keywords_sector = {
        "software": ["software", "application", "program"],
        "biotechnology": ["gene", "biotech", "protein"],
        "electronics": ["circuit", "chip", "semiconductor"],
        # Add more sectors and keywords as needed
    }
    for sector, keywords in keywords_sector.items():
        if any(keyword in title for keyword in keywords):
            return sector
    return "unknown"

def determine_industry(title):
    keywords_industry = {
        "healthcare": ["medical", "health", "surgery"],
        "automotive": ["vehicle", "car", "engine"],
        "telecom": ["communication", "network", "signal"],
        # Add more industries and keywords as needed
    }
    for industry, keywords in keywords_industry.items():
        if any(keyword in title for keyword in keywords):
            return industry
    return "unknown"

sector_udf = udf(determine_sector, StringType())
industry_udf = udf(determine_industry, StringType())

df = df.withColumn("technology_sector", sector_udf(df.tokenized_title))
df = df.withColumn("industry", industry_udf(df.tokenized_title))

# Step 5: Save the Updated DataFrame
output_path = "updated_data_with_sectors_industries.xlsx"
df.write.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .save(output_path)


+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+-----------------+
|              id|               title|                           assignee|                inventor/author|priority date|filing/creation date|publication date|grant date|technology_sector|
+----------------+--------------------+-----------------------------------+-------------------------------+-------------+--------------------+----------------+----------+-----------------+
|  CN-114151200-A|            Igniter |         中国航发四川燃气涡轮研究院|  魏豪, 时远, 何鹏, 赵弦, 袁...|   2021-10-20|          2021-10-20|      2022-03-08|      NULL|          Telecom|
|   US-9869277-B2|System, method an...|               Foret Plasma Labs...|                     Todd Foret|   2008-02-12|          2014-05-28|      2018-01-16|2018-01-16|      Electronics|
|US-2021215100-A1|Blow down impinge...|               Rolls-Royce North...|  

In [ ]:
df_with_technology_sector.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- assignee: string (nullable = true)
 |-- inventor/author: string (nullable = true)
 |-- priority date: string (nullable = true)
 |-- filing/creation date: string (nullable = true)
 |-- publication date: string (nullable = true)
 |-- grant date: string (nullable = true)
 |-- technology_sector: string (nullable = true)



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import random

# Initialize Spark session
spark = SparkSession.builder \
    .appName("our first session") \
    .config("spark.sql.sessionEncoding", "UTF-8") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv("googlePatents.csv", header=True, encoding="UTF-8")



# Add a new column with random technology sectors
df_with_technology_sector = df.withColumn("technology_sector", random_technology_sector_udf())

# Select only the "id" and "technology_sector" columns
df_selected = df_with_technology_sector.select("id", "technology_sector")

# Display the DataFrame
df_selected.show()


+----------------+-----------------+
|              id|technology_sector|
+----------------+-----------------+
|  CN-114151200-A|         Software|
|   US-9869277-B2|          Telecom|
|US-2021215100-A1|      Electronics|
|   US-7966830-B2|          Telecom|
|    GB-2602936-A|    Biotechnology|
|   EP-2086834-B1|    Biotechnology|
|  US-11572840-B2|       Automotive|
|  CN-112855343-A|      Electronics|
|  CN-114109649-B|       Healthcare|
|  CN-108839807-A|         Software|
|KR-20220002857-A|         Software|
|US-2019293024-A1|          Telecom|
|   US-9291124-B2|         Software|
|   RU-2462397-C2|          Telecom|
|  US-11885260-B2|    Biotechnology|
|   US-8434292-B2|    Biotechnology|
|   US-7690192-B2|         Software|
|   EP-2074324-B1|       Healthcare|
|    US-2655786-A|    Biotechnology|
|  US-11542878-B2|       Automotive|
+----------------+-----------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import random

# Initialize Spark session
spark = SparkSession.builder \
    .appName("our first session") \
    .config("spark.sql.sessionEncoding", "UTF-8") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv("googlePatents.csv", header=True, encoding="UTF-8")

# Define a list of possible technology sectors
technology_sectors = ["Software", "Biotechnology", "Electronics", "Healthcare", "Automotive", "Telecom"]

# Create a UDF to assign random technology sectors
def random_technology_sector():
    return random.choice(technology_sectors)

random_technology_sector_udf = udf(random_technology_sector, StringType())

# Add a new column with random technology sectors
df_with_technology_sector = df.withColumn("technology_sector", random_technology_sector_udf())

# Select only the "id" and "technology_sector" columns
df_selected = df_with_technology_sector.select("id", "technology_sector")

# Save the resulting DataFrame as a CSV file
output_path = "patents_with_technology_sector.csv"
df_selected.write.csv(output_path, header=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/My Drive/patents_with_technology_sector.csv"
df_selected.write.csv(output_path, header=True)


Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify output path
output_path = "/content/drive/My Drive/patents_with_technology_sector.xlsx"

# Convert Spark DataFrame to Pandas DataFrame
df_pandas = df_selected.toPandas()

# Save Pandas DataFrame to XLSX file
df_pandas.to_excel(output_path, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
